In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import datetime

In [3]:
conn = sqlite3.connect('database.db')

c = conn.cursor()

In [2]:
# tik jei norima scrapinti is naujo

c.execute("""DROP TABLE Votes""")
c.execute("""DROP TABLE MPs""")
c.execute("""DROP TABLE Parties""")
c.execute("""DROP TABLE Motions""")
c.execute("""DROP TABLE Affiliations""")

conn.commit()

NameError: name 'c' is not defined

In [4]:
# MPs table tiesiog sarasas visu MPs vardu ir ID
c.execute("""
CREATE TABLE IF NOT EXISTS MPs
(Name TEXT,
 MP_id INTEGER PRIMARY KEY) 
        """)

# Sarasas visu balsavimu
c.execute("""
CREATE TABLE IF NOT EXISTS Motions
(Term_id INTEGER,
 Session_id INTEGER,
 Meeting_id INTEGER,
 Motion_id INTEGER PRIMARY KEY,
 Motion_type TEXT,
 Motion TEXT)
        """)

# Sarasas kiekvieno balso (kiekvienas balsas yra unikali kombinacija motion_id ir mp_id)
c.execute("""
CREATE TABLE IF NOT EXISTS Votes
(MP_id INTEGER NOT NULL,
 Motion_id INTEGER NOT NULL,
 Vote INTEGER,
 PRIMARY KEY (MP_id, Motion_id)) 
        """)

# Sarasas partiju
c.execute("""
CREATE TABLE IF NOT EXISTS Parties
(Name TEXT PRIMARY KEY)
""")

# Sarasas afiliaciju partijoms; atskirtas nuo MPs table, nes skirtingose kadencijose tas pats MP gali priklausyti skirtingoms partijoms
c.execute("""
CREATE TABLE IF NOT EXISTS Affiliations
(Party_id INTEGER,
Term_id INTEGER,
MP_id INTEGER,
PRIMARY KEY (Term_id, MP_id))""")

conn.commit()


In [75]:
c.close()
conn.close()

In [11]:
# uzsipildom MPs, Parties ir Affiliations tables

kadencijos_url = 'http://apps.lrs.lt/sip/p2b.ad_seimo_kadencijos'
kadencijos_page = requests.get(kadencijos_url)
kadencijos_soup = BeautifulSoup(kadencijos_page.text, 'html5lib')
kadencijos = kadencijos_soup.find_all('seimokadencija')
kadencijos_ids = [kadencija.attrs['kadencijos_id'] for kadencija in kadencijos]

for kadencijos_id in kadencijos_ids:
    
    kadencijos_id_int = int(kadencijos_id)
    
    seimo_nariai_url = 'http://apps.lrs.lt/sip/p2b.ad_seimo_nariai?kadencijos_id=' + kadencijos_id
    seimo_nariai_page = requests.get(seimo_nariai_url)
    seimo_nariai_soup = BeautifulSoup(seimo_nariai_page.text, 'html5lib')
    seimo_nariai = seimo_nariai_soup.find_all('seimonarys')
    
    for seimo_narys in seimo_nariai:
        
        mp_id_int = int(seimo_narys.attrs['asmens_id'])
        name = seimo_narys.attrs['vardas'] + ' ' + seimo_narys.attrs['pavardė']
        partija = seimo_narys.attrs['iškėlusi_partija']
        
        c.execute("""
        INSERT OR IGNORE INTO Parties
        (Name)
        VALUES (?)
        """, (partija,))
        
        c.execute("""
        SELECT rowid FROM Parties
        WHERE Name = ?""", (partija,))

        party_id = c.fetchall()[0][0]
        
        c.execute("""
        INSERT OR IGNORE INTO Affiliations
        (Party_id, Term_id, MP_id)
        VALUES (?,?,?)
        """,
        (party_id, kadencijos_id_int, mp_id_int))
        
        c.execute("""
        INSERT OR IGNORE INTO MPs
        (Name, MP_id)
        VALUES (?,?)""", (name, mp_id_int))
        
conn.commit()
        
        
    

In [83]:
balsai_int = {
    'Prieš': -1,
    'Už': 1,
    'Susilaikė': 0,
    '': None
}

# kadencija -> sesija -> posedis -> klausimas -> balsavimas uz klausima
kadencijos_url = 'http://apps.lrs.lt/sip/p2b.ad_seimo_kadencijos'
kadencijos_page = requests.get(kadencijos_url)
kadencijos_soup = BeautifulSoup(kadencijos_page.text, 'html5lib')
kadencijos = kadencijos_soup.find_all('seimokadencija')
kadencijos_ids = [kadencija.attrs['kadencijos_id'] for kadencija in kadencijos]

# pirmos 4 kadencijos inconsistently savo duomenis sudejusios, tai nuo penktos pradejau rinkti
for kadencijos_id in kadencijos_ids[4:]:
    
    sesijos_url = 'http://apps.lrs.lt/sip/p2b.ad_seimo_sesijos?kadencijos_id=' + kadencijos_id
    sesijos_page = requests.get(sesijos_url)
    sesijos_soup = BeautifulSoup(sesijos_page.text, 'html5lib')
    sesijos = sesijos_soup.find_all('seimosesija')
    sesijos_ids = [sesija.attrs['sesijos_id'] for sesija in sesijos]
    if kadencijos_id == '9':
        sesijos_ids = ['131']
    for sesijos_id in sesijos_ids:
        
        posedziai_url = 'http://apps.lrs.lt/sip/p2b.ad_seimo_posedziai?sesijos_id=' + sesijos_id
        posedziai_page = requests.get(posedziai_url)
        posedziai_soup = BeautifulSoup(posedziai_page.text, 'html5lib')
        posedziai_elements = posedziai_soup.find_all('seimoposėdis')

        # hackas paskutinei sesijai, nes sarase yra dar neivykusiu sesiju
        #p = []
        #for p_element in posedziai_elements:
        #    if int(p_element.attrs['numeris']) < 334:
        #        p.append(p_element)
        #posedziai_elements = p
                
        posedziai = [{
            'id': posedis.attrs['posėdžio_id'],
            'start_date': datetime.datetime.fromisoformat(posedis.attrs['pradžia']).date()
        } for posedis in posedziai_elements]

        # kartais vakariniame posedyje svarstoma rytinio posedzio klausimai, ir vice versea. tai reikia sujungti
        # labai clunky kodas, bet nesugalvojau nieko elegenatiskiau
        
        posedziai_by_date = []
        
        i = 0
        while i < len(posedziai):
            if i == len(posedziai) - 1 or posedziai[i]['start_date'] != posedziai[i+1]['start_date']:
                posedziai_by_date.append({
                    'id': [posedziai[i]['id']],
                    'start_date': posedziai[i]['start_date']
                })
                i += 1
            else:
                posedziai_by_date.append({
                    'id': [posedziai[i]['id'], posedziai[i+1]['id']],
                    'start_date': posedziai[i]['start_date']
                })
                i+= 2
        
        for posedziai in posedziai_by_date:

            klausimai_eiga = []
            klausimai_darbotvarke = []
            
            for posedis_id in posedziai['id']:
                
                posedis_eiga_url = 'http://apps.lrs.lt/sip/p2b.ad_sp_eiga?posedzio_id=' + posedis_id 
                posedis_eiga_page = requests.get(posedis_eiga_url)
                posedis_eiga_soup = BeautifulSoup(posedis_eiga_page.text, 'html5lib')
                klausimai_eiga_soup = posedis_eiga_soup.find_all('eigosklausimas')
                
                klausimai_eiga += [{
                    'posedzio_id': posedis_id,
                    'id': klausimas.attrs['svarstomo_klausimo_id'],
                    'name': klausimas.attrs['pavadinimas'],
                    'type': klausimas.attrs['stadija'],
                    'no': klausimas.attrs['numeris']
                                  } for klausimas in klausimai_eiga_soup]
                
                posedis_darbotvarke_url = 'http://apps.lrs.lt/sip/p2b.ad_sp_darbotvarke?posedzio_id=' + posedis_id 
                posedis_darbotvarke_page = requests.get(posedis_darbotvarke_url)
                posedis_darbotvarke_soup = BeautifulSoup(posedis_darbotvarke_page.text, 'html5lib')
                klausimai_darbotvarke_soup = posedis_darbotvarke_soup.find_all('darbotvarkėsklausimas')
    
                klausimai_darbotvarke += [{
                    'name': klausimas.attrs['pavadinimas'],
                    'no': klausimas.attrs['numeris']
                                  } for klausimas in klausimai_darbotvarke_soup]
                
            for klausimas in klausimai_eiga:

                if len(klausimas['name']) >= 14 and klausimas['name'][0:14] == "Klausimų grupė":

                    # surandi visus klausimus darbotvarkeje, kurie atitinka eigos klausima

                    sub_klausimai = klausimas['name'][16:].split(", ")
                    klausimas['name'] = "Klausimų grupė: "

                    for klausimas_d in klausimai_darbotvarke:

                        no = klausimas_d['no']
                        no = no[:-1] if no != '' and no[-1] == '.' else no
                        
                        if no in sub_klausimai:

                            klausimas['name'] = klausimas['name'] + klausimas_d['name'] + " + "

                    klausimas['name'] = klausimas['name'][:-2]

                svarstymas_url = 'http://apps.lrs.lt/sip/p2b.ad_sp_klausimo_svarstymo_eiga?svarstomo_klausimo_id=' + klausimas['id']
                svarstymas_page = requests.get(svarstymas_url)
                svarstymas_soup = BeautifulSoup(svarstymas_page.text, 'html5lib')
                ivykiai = svarstymas_soup.find_all('svarstymoeigosĮvykis')

                for ivykis in ivykiai:

                    if ivykis.attrs['įvykio_tipas'] == 'Balsavimas':
                        
                        balsavimas_id = ivykis.attrs['balsavimo_id']
                        balsavimas_url = 'http://apps.lrs.lt/sip/p2b.ad_sp_balsavimo_rezultatai?balsavimo_id=' + balsavimas_id
                        balsavimas_page = requests.get(balsavimas_url)
                        balsavimas_soup = BeautifulSoup(balsavimas_page.text, 'html5lib')
                        balsai = balsavimas_soup.find_all('individualusbalsavimorezultatas')
                        
                        if balsai != []:
                            
                            c.execute("""
                            INSERT OR IGNORE INTO Motions
                            (Term_id, Session_id, Meeting_id, Motion_id, Motion_type, Motion)
                            VALUES (?,?,?,?,?,?)""", (int(kadencijos_id), int(sesijos_id), int(posedis_id), int(klausimas['id']), klausimas['type'], klausimas['name']))
                    
                            for balsas in balsai:
                                
                                balsas_type = balsas.attrs['kaip_balsavo']
                                mp_id = int(balsas.attrs['asmens_id'])
                                klausimas_id_int = int(klausimas['id'])
                                balsas_int = balsai_int[balsas_type]
                                
                                c.execute("""
                                INSERT OR IGNORE INTO Votes
                                (MP_id, Motion_id, Vote)
                                VALUES (?,?,?)
                                """, (mp_id, klausimas_id_int, balsas_int))
                                
                                conn.commit()
                        
                conn.commit()
                        
                        
                        
                        
            
        


In [87]:
# pasitikrinimui

c.execute("""
SELECT * FROM Motions
WHERE Motion LIKE "%partneryst%"
LIMIT 10""")

ans = c.fetchall()

print(ans)

[(9, 131, -501873, -47839, 'Priėmimas', 'Įstatymo „Dėl Europos Sąjungos bei jos valstybių narių ir Tailando Karalystės pagrindų susitarimo dėl visapusiškos partnerystės ir bendradarbiavimo ratifikavimo“ projektas (Nr. XIVP-3043(2))'), (9, 131, -501873, -47838, 'Priėmimas', 'Įstatymo "Dėl Europos Sąjungos bei jos valstybių narių ir Malaizijos Vyriausybės pagrindų susitarimo dėl partnerystės ir bendradarbiavimo ratifikavimo" projektas (Nr. XIVP-3042(2))'), (9, 131, -501871, -47819, 'Svarstymas', 'Įstatymo „Dėl Europos Sąjungos bei jos valstybių narių ir Tailando Karalystės pagrindų susitarimo dėl visapusiškos partnerystės ir bendradarbiavimo ratifikavimo“ projektas (Nr. XIVP-3043(2))'), (9, 131, -501871, -47818, 'Svarstymas', 'Įstatymo "Dėl Europos Sąjungos bei jos valstybių narių ir Malaizijos Vyriausybės pagrindų susitarimo dėl partnerystės ir bendradarbiavimo ratifikavimo" projektas (Nr. XIVP-3042(2))'), (9, 131, -501859, -47535, 'Pateikimas', 'Įstatymo „Dėl Europos Sąjungos bei jos v